In [1]:
import pandas as pd
import numpy as np

import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve
from catboost import Pool, CatBoostClassifier

from scipy.stats import pearsonr, chi2_contingency
from itertools import combinations
from statsmodels.stats.proportion import proportion_confint

There are 2 million rows in total, in order to reduce the pricessing times here only selecting the loans issued in 2018.

In [2]:
data = pd.read_csv(
    'accepted_2007_to_2018Q4.csv',
    parse_dates=['issue_d'], infer_datetime_format=True)
data = data[(data.issue_d >= '2018-01-01 00:00:00') & (data.issue_d < '2019-01-01 00:00:00')]
data = data.reset_index(drop=True)
data.head()

/Users/bcchochoao/opt/anaconda2/envs/Notebook/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,130954621,NaN,5000.0,5000.0,5000.0,36 months,20.39,186.82,D,D4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,130964697,NaN,15000.0,15000.0,15000.0,36 months,9.92,483.45,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,130955326,NaN,11200.0,11200.0,11200.0,60 months,30.79,367.82,G,G1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,130504052,NaN,25000.0,25000.0,25000.0,60 months,21.85,688.35,D,D5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,130956066,NaN,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,A4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
browse_notes = pd.read_excel('LCDataDictionary.xlsx',
                             sheet_name=1)
browse_notes.head()

,BrowseNotesFile,Description
0,acceptD,The date which the borrower accepted the offer
1,accNowDelinq,The number of accounts on which the borrower i...
2,accOpenPast24Mths,Number of trades opened in past 24 months.
3,addrState,The state provided by the borrower in the loan...
4,all_util,Balance to credit limit on all trades


The 'LCDataDictionary.xlsx' contains the list of geatures that are visible to investors.

In [4]:
browse_feat = browse_notes['BrowseNotesFile'].dropna().values
browse_feat = [re.sub('(?<![0-9_])(?=[A-Z0-9])', '_', x).lower().strip() for x in browse_feat]

The missing rows are removed from 'browse_notes' and saved in 'browse_feat'

In [5]:
data_feat = data.columns.values
np.setdiff1d(browse_feat, data_feat)

array(['accept_d', 'credit_pull_d', 'effective_int_rate', 'exp_d',
       'exp_default_rate', 'ils_exp_d', 'is_inc_v', 'list_d', 'msa',
       'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
       'mths_since_recent_loan_delinq', 'review_status',
       'review_status_d', 'service_fee_rate', 'verified_status_joint'],
      dtype='<U35')

In [6]:
np.setdiff1d(data_feat, browse_feat)

array(['collection_recovery_fee', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'deferral_term', 'funded_amnt_inv',
       'hardship_amount', 'hardship_dpd', 'hardship_end_date',
       'hardship_flag', 'hardship_last_payment_amount', 'hardship_length',
       'hardship_loan_status', 'hardship_payoff_balance_amount',
       'hardship_reason', 'hardship_start_date', 'hardship_status',
       'hardship_type', 'issue_d', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt',
       'last_pymnt_d', 'loan_status', 'mo_sin_old_il_acct',
       'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
       'next_pymnt_d', 'orig_projected_additional_accrued_interest',
       'out_prncp', 'out_prncp_inv', 'payment_plan_start_date',
       'policy_code', 'pymnt_plan', 'recoveries', 'settlement_amount',
       'settlement_date', 'settlement_percentage', 'settlement_status',
       'settlement_term', 'total_pymnt', 'total_pymnt_inv',
       'total

In [7]:
wrong = ['is_inc_v', 'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
         'mths_since_recent_loan_delinq', 'verified_status_joint']
correct = ['verification_status', 'mths_since_recent_inq', 'mo_sin_old_il_acct',
           'mths_since_recent_bc_dlq', 'verification_status_joint']

browse_feat = np.setdiff1d(browse_feat, wrong)
browse_feat = np.append(browse_feat, correct)

We found that some features are named differently but represent the same thing. Thus, I name one of them as correct, and the rest same features wrong. Then, I remove the wrong ones and only save the correct ones.

In [8]:
avail_feat = np.intersect1d(browse_feat, data_feat)
X = data[avail_feat].copy()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495242 entries, 0 to 495241
Columns: 109 entries, acc_now_delinq to zip_code
dtypes: float64(89), object(20)
memory usage: 411.8+ MB


All marching features are saved in avail_feat list. Then, create a new DataFrame to restore data.
Create a DataFrame as a copy is good to avoid hidden chained assignments.

In [9]:
X.select_dtypes('object').head()

,addr_state,application_type,desc,disbursement_method,earliest_cr_line,emp_length,emp_title,grade,home_ownership,id,initial_list_status,purpose,sec_app_earliest_cr_line,sub_grade,term,title,url,verification_status,verification_status_joint,zip_code
0,OK,Individual,NaN,Cash,Jan-2009,8 years,General Manager,D,RENT,130954621,w,other,NaN,D4,36 months,Other,https://lendingclub.com/browse/loanDetail.acti...,Verified,NaN,740xx
1,FL,Individual,NaN,Cash,Jul-1998,2 years,IT Director,B,OWN,130964697,w,debt_consolidation,NaN,B2,36 months,Debt consolidation,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,337xx
2,NH,Joint App,NaN,Cash,Jul-2007,< 1 year,Client services,G,RENT,130955326,w,medical,Feb-2005,G1,60 months,Medical expenses,https://lendingclub.com/browse/loanDetail.acti...,Not Verified,Not Verified,030xx
3,AL,Individual,NaN,Cash,Mar-1995,10+ years,Asphalt Supervisor,D,MORTGAGE,130504052,w,debt_consolidation,NaN,D5,60 months,Debt consolidation,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,361xx
4,WA,Individual,NaN,Cash,Jan-1998,9 years,Scale Technician,A,RENT,130956066,w,major_purchase,NaN,A4,36 months,Major purchase,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,988xx


Now, check the categorical features and see if we can transfer them to other types.

The 'earliest_cr_line' and 'sec_app_earliest_cr_line' are dates and their type should be changed to datetime. Later, then will be transformed to ordinal numeric features by the machine learning model.

In [10]:
X['earliest_cr_line'] = pd.to_datetime(X['earliest_cr_line'], infer_datetime_format=True)
X['sec_app_earliest_cr_line'] = pd.to_datetime(X['sec_app_earliest_cr_line'], infer_datetime_format=True)

In [11]:
X['emp_length'] = X['emp_length'].replace({'< 1 year': '0 years', '10+ years': '11 years'})
X['emp_length'] = X['emp_length'].str.extract('(\d+)').astype('float')
X['id'] = X['id'].astype('float')

The emp_length and id are numeric, we need to transfer them to float type. And, change <1 years to 0 year and change 10+ years to 11 years.

In [ ]:
#Dealing with Missing Values:

In [12]:
nan_mean = X.isna().mean()
nan_mean = nan_mean[nan_mean != 0].sort_values()
nan_mean
#Here is the columns with missing values and their ratio to the total number of rows:

pct_tl_nvr_dlq                         0.000004
avg_cur_bal                            0.000081
all_util                               0.000260
revol_util                             0.001195
dti                                    0.002286
mths_since_recent_bc                   0.012515
bc_open_to_buy                         0.013303
percent_bc_gt_75                       0.013319
bc_util                                0.013737
num_tl_120dpd_2m                       0.025046
mo_sin_old_il_acct                     0.037174
mths_since_rcnt_il                     0.037174
emp_length                             0.084781
emp_title                              0.110368
mths_since_recent_inq                  0.123788
il_util                                0.163201
mths_since_last_delinq                 0.558620
mths_since_recent_revol_delinq         0.711878
mths_since_last_major_derog            0.768128
mths_since_recent_bc_dlq               0.801895
sec_app_chargeoff_within_12_mths       0

The features 'desc' and 'member_id' are completely empty, so I remove them.

In [13]:
X = X.drop(['desc', 'member_id'], axis=1, errors='ignore')

1.Categorical features: the missing values will be filled with an empty string " ".
2.Numeric features: the missing values should be filled using the maximum value of the respective columns, so these features are placed in the list 'filled_max'.
3.Rest of numerix features the missing values should be filled using the minimum values of the respective columns, so these features are placed in the list 'fill_min'.

In [14]:
fill_empty = ['emp_title', 'verification_status_joint']
fill_max = ['bc_open_to_buy', 'mo_sin_old_il_acct', 'mths_since_last_delinq',
            'mths_since_last_major_derog', 'mths_since_last_record',
            'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
            'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
            'pct_tl_nvr_dlq','sec_app_mths_since_last_major_derog']
fill_min = np.setdiff1d(X.columns.values, np.append(fill_empty, fill_max))

X[fill_empty] = X[fill_empty].fillna('')
X[fill_max] = X[fill_max].fillna(X[fill_max].max())
X[fill_min] = X[fill_min].fillna(X[fill_min].min())

In [ ]:
#Multicollinearity:
#The highly correlated features decrease importances of each other and can make feature analysis more difficult.
#After calculating correlations, then remove the features with high correlation coefficients

In [15]:
num_feat = X.select_dtypes('number').columns.values
X[num_feat].nunique().sort_values()

num_tl_120dpd_2m                   1
acc_now_delinq                     2
num_tl_30dpd                       2
inq_last_6mths                     6
sec_app_inq_last_6mths             7
                               ...  
total_il_high_credit_limit    128691
total_bal_ex_mort             139900
tot_cur_bal                   254794
tot_hi_cred_lim               266282
id                            495242
Length: 90, dtype: int64

The feature 'num_t1_120dpd_2m' has only one value (a constant) and can be removed. The feature 'id' has a unique value for each row and should also be removed, otherwise the model will over fit.

In [16]:
X = X.drop(['num_tl_120dpd_2m', 'id'], axis=1, errors='ignore')

In [17]:
#For all pairs of the numeric features: 
#I calculate their Pearson's R correlation coefficient and store it in corr_num_feat.
num_feat = X.select_dtypes('number').columns.values
comb_num_feat = np.array(list(combinations(num_feat, 2)))
corr_num_feat = np.array([])
for comb in comb_num_feat:
    corr = pearsonr(X[comb[0]], X[comb[1]])[0]
    corr_num_feat = np.append(corr_num_feat, corr)

In [18]:
#The highly correlated pairs with the absolute value of their correlation coefficient ≥0.9 are printed below.
high_corr_num = comb_num_feat[np.abs(corr_num_feat) >= 0.9]
high_corr_num

array([['acc_now_delinq', 'num_tl_30dpd'],
       ['fico_range_high', 'fico_range_low'],
       ['funded_amnt', 'installment'],
       ['funded_amnt', 'loan_amnt'],
       ['installment', 'loan_amnt'],
       ['mo_sin_old_il_acct', 'mths_since_rcnt_il'],
       ['num_actv_rev_tl', 'num_rev_tl_bal_gt_0'],
       ['num_sats', 'open_acc'],
       ['sec_app_fico_range_high', 'sec_app_fico_range_low'],
       ['sec_app_num_rev_accts', 'sec_app_open_acc'],
       ['tot_cur_bal', 'tot_hi_cred_lim'],
       ['total_bal_ex_mort', 'total_bal_il'],
       ['total_bal_il', 'total_il_high_credit_limit']], dtype='<U35')

In [19]:
#The first feature (chosen arbitrarily) from each highly correlated feature pair is then removed.
X = X.drop(np.unique(high_corr_num[:, 0]), axis=1, errors='ignore')

In [20]:
#Then I print out the number of unique values for categorical features.
cat_feat = X.select_dtypes('object').columns.values
X[cat_feat].nunique().sort_values()

application_type                  2
disbursement_method               2
initial_list_status               2
term                              2
verification_status               3
home_ownership                    4
verification_status_joint         4
grade                             7
title                            12
purpose                          13
sub_grade                        35
addr_state                       50
zip_code                        897
emp_title                    129450
url                          495242
dtype: int64

In [23]:
#Remove both 'url' and 'emp_title' since they have too many unique values. It can avoid overfitting.
X = X.drop(['url','emp_title'], axis=1, errors = 'ignore')

In [24]:
#For all pairs of the categorical features :

I calculate the Cramer's V correlation coefficient that is expressed through the chi-square statistic  𝜒2
χ
2
  of the contingency table,where  𝑛
n
  is the sum of all elements in the contingency table,  𝐾1
K
1
  and  𝐾2
K
2
  are the dimensions of the contingency table. Note that Pearson's R correlation coefficient isn't applicable to categorical features and shouldn't be used.

In [22]:
cat_feat = X.select_dtypes('object').columns.values
comb_cat_feat = np.array(list(combinations(cat_feat, 2)))
corr_cat_feat = np.array([])
for comb in comb_cat_feat:
    table = pd.pivot_table(X, values='loan_amnt', index=comb[0], columns=comb[1], aggfunc='count').fillna(0)
    corr = np.sqrt(chi2_contingency(table)[0] / (table.values.sum() * (np.min(table.shape) - 1) ) )
    corr_cat_feat = np.append(corr_cat_feat, corr)

In [25]:
#The highly correlated pairs with the absolute value of their correlation coefficient ≥0.9 are printed below.
high_corr_cat = comb_cat_feat[corr_cat_feat >= 0.9]
high_corr_cat

array([['addr_state', 'zip_code'],
       ['application_type', 'verification_status_joint'],
       ['grade', 'sub_grade'],
       ['purpose', 'title']], dtype='<U25')

This time I remove the second feature from each highly correlated feature pair in order to keep the feature grade. It will be used later to select high risk / high interest loans.

In [26]:
X = X.drop(np.unique(high_corr_cat[:, 1]), axis=1, errors='ignore')

In [28]:
#Target Features:

The target feature for this dataset is an indicator if the loan is good (1) or bad (0). To identify good loans, I use their loan statuses and print their counts below. The description for each status is provided by the Lending Club:

Current: Loan is up to date on all outstanding payments.
In Grace Period: Loan is past due but within the 15-day grace period.
Late (16-30): Loan has not been current for 16 to 30 days.
Late (31-120): Loan has not been current for 31 to 120 days.
Fully paid: Loan has been fully repaid, either at the expiration of the 3- or 5-year year term or as a result of a prepayment.
Default: Loan has not been current for an extended period of time.
Charged Off: Loan for which there is no longer a reasonable expectation of further payments.

In [29]:
data['loan_status'].value_counts()

Current               427181
Fully Paid             47444
Charged Off             8867
Late (31-120 days)      7221
In Grace Period         2919
Late (16-30 days)       1603
Default                    7
Name: loan_status, dtype: int64

The loans with the statuses Current and Fully Paid are definitely considered good. The loans with the statuses In Grace Period can be considered good or not depending on strictness of the investor. In this project I consider them as good loans. All the other statuses are considered as bad loans. Note that this dataset is highly imbalanced with the minority class being 0.037 of the majority class.

In [30]:
y = data['loan_status'].copy()
y = y.isin(['Current', 'Fully Paid', 'In Grace Period']).astype('int')
y.value_counts()

1    477544
0     17698
Name: loan_status, dtype: int64

In [ ]:
# Modelling: